## Week 5 Homework
### Spark

### Q1

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('week_5_hmw') \
    .getOrCreate()

22/03/01 19:55:56 WARN Utils: Your hostname, DESKTOP-FHCRPTH resolves to a loopback address: 127.0.1.1; using 172.29.215.187 instead (on interface eth0)
22/03/01 19:55:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/01 19:55:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.version

'3.0.3'

### Q2

In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 00:01:16--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.104.68
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.104.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  52.6MB/s    in 12s     

2022-03-01 00:01:28 (58.4 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [4]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [6]:
df = df.repartition(24)

In [8]:
df.write.parquet('fhvhv/2021/02/', mode='overwrite')

22/03/01 19:58:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 19:58:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/03/01 19:58:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
22/03/01 19:58:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
22/03/01 19:58:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
22/03/01 19:58:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
22/03/01 19:58:11 WARN MemoryManager: Total allocation exceeds 95.

In [12]:
!du -h

202M	./fhvhv/2021/02
202M	./fhvhv/2021
202M	./fhvhv
902M	.


### Q3

In [23]:
df = spark.read.parquet('fhvhv/2021/02')

In [15]:
df.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B02878', pickup_datetime=datetime.datetime(2021, 2, 3, 13, 34, 55), dropoff_datetime=datetime.datetime(2021, 2, 3, 13, 50, 32), PULocationID=170, DOLocationID=143, SR_Flag=None)

In [16]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [17]:
from pyspark.sql import functions as F

In [18]:
df.groupBy(F.to_date(df.pickup_datetime)).count().show()

+--------------------------+------+
|to_date(`pickup_datetime`)| count|
+--------------------------+------+
|                2021-02-15|367170|
|                2021-02-02|302785|
|                2021-02-26|500049|
|                2021-02-21|399763|
|                2021-02-05|478766|
|                2021-02-10|403476|
|                2021-02-01|176946|
|                2021-02-06|470555|
|                2021-02-19|459887|
|                2021-02-20|497072|
|                2021-02-08|398445|
|                2021-02-09|398476|
|                2021-02-11|428288|
|                2021-02-17|415856|
|                2021-02-25|436556|
|                2021-02-27|509383|
|                2021-02-24|412556|
|                2021-02-18|372878|
|                2021-02-14|460661|
|                2021-02-12|469162|
+--------------------------+------+
only showing top 20 rows



### Q4
Ref: https://sparkbyexamples.com/pyspark/pyspark-timestamp-difference-seconds-minutes-hours/#:~:text=Timestamp%20difference%20in%20PySpark%20can,the%20minute%20difference%20and%20finally

In [24]:
df = df.withColumn('DiffInSeconds',df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long"))

In [26]:
df.orderBy(df.DiffInSeconds.desc()).show(3)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|DiffInSeconds|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|         247|          41|   null|        75540|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|         242|         254|   null|        57221|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|         188|          55|   null|        44039|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------+
only showing top 3 rows



### Q5

In [27]:
df.groupBy('dispatching_base_num').count().show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02876| 215693|
|              B03136|   1741|
|              B02877| 198938|
|              B02869| 429720|
|              B02883| 251617|
|              B02835| 189031|
|              B02884| 244963|
|              B02880| 115716|
|              B02878| 305185|
|              B02836| 128978|
|              B02872| 882689|
|              B02512|  41043|
|              B02867| 200530|
|              B02866| 311089|
|              B02871| 312364|
|              B02889| 138762|
|              B02844|   3502|
|              B02510|3233664|
|              B02888| 169167|
|              B02682| 303255|
+--------------------+-------+
only showing top 20 rows



## Q6

In [28]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'DiffInSeconds']

In [31]:
df.groupBy('PULocationID', 'DOLocationID').count().sort(F.desc('count')).show()

+------------+------------+-----+
|PULocationID|DOLocationID|count|
+------------+------------+-----+
|          76|          76|45041|
|          26|          26|37329|
|          39|          39|28026|
|          61|          61|25976|
|          14|          14|17934|
|           7|           7|14688|
|         129|         129|14688|
|          42|          42|14481|
|          37|          37|14424|
|          89|          89|13976|
|         216|         216|13716|
|          35|          35|12829|
|         132|         265|12542|
|         188|          61|11814|
|          95|          95|11548|
|          36|          37|11491|
|          37|          36|11487|
|          61|         188|11462|
|          61|         225|11342|
|         188|         188|11308|
+------------+------------+-----+
only showing top 20 rows



In [32]:
result = df.groupBy('PULocationID', 'DOLocationID').count().sort(F.desc('count'))

In [37]:
# load in zone_lookup
zone = spark.read \
    .option("header", True) \
    .csv("taxi+_zone_lookup.csv")

In [38]:
zone.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [39]:
zone = zone.select('LocationID', 'Zone')

In [44]:
# join
result.join(zone, result.PULocationID == zone.LocationID, 'inner') \
    .drop('LocationID') \
    .withColumnRenamed('Zone', 'PU_zone') \
    .join(zone, result.DOLocationID == zone.LocationID, 'inner') \
    .drop('LocationID') \
    .withColumnRenamed('Zone', 'DO_zone') \
    .sort(F.desc('count')) \
    .show()

+------------+------------+-----+--------------------+--------------------+
|PULocationID|DOLocationID|count|             PU_zone|             DO_zone|
+------------+------------+-----+--------------------+--------------------+
|          76|          76|45041|       East New York|       East New York|
|          26|          26|37329|        Borough Park|        Borough Park|
|          39|          39|28026|            Canarsie|            Canarsie|
|          61|          61|25976| Crown Heights North| Crown Heights North|
|          14|          14|17934|           Bay Ridge|           Bay Ridge|
|           7|           7|14688|             Astoria|             Astoria|
|         129|         129|14688|     Jackson Heights|     Jackson Heights|
|          42|          42|14481|Central Harlem North|Central Harlem North|
|          37|          37|14424|      Bushwick South|      Bushwick South|
|          89|          89|13976|Flatbush/Ditmas Park|Flatbush/Ditmas Park|
|         21